![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to analyze car rental customer satisfaction and offer recommendation.

**Note:** Please note that for the watsonx challenge, please run these notebooks in IBM Cloud and not on on your laptop/desktop.

This notebook contains the steps and code to demonstrate support of text sentiment analysis in Watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

<a id="setup"></a>
## Set up the environment

### Install and import the dependecies

In [2]:
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install ibm-watson-machine-learning==1.0.312 | tail -n 1

**Note:** Please restart the notebook kernel to pick up proper version of packages installed above.

In [3]:
import os, getpass
from pandas import read_csv

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. Instructions have been provided to generate IBM Cloud API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [4]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

Please enter your api key (hit enter): ········


### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. When you run notebook on IBM Cloud, project in which it runs is saved as environment variable PROJECT_ID.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [5]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [6]:
filename_test = 'https://watsonx-gsi-challenge.s3.jp-tok.cloud-object-storage.appdomain.cloud/track1/test.csv'
filename_train = 'https://watsonx-gsi-challenge.s3.jp-tok.cloud-object-storage.appdomain.cloud/track1/train.csv'

test_data = read_csv(filename_test)
train_data = read_csv(filename_train)

In [7]:
train_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2944,Female,M,2,41.92,Active,No,Customer service was friendly and helpful.,1,Service: Knowledge,NaN
1,1119,Female,M,2,33.60,Active,Yes,Customer service was good at MSP airport and t...,1,Service: Knowledge,NaN
2,0,Male,M,0,51.00,Inactive,Yes,I do not understand why I have to pay additio...,0,Product: Pricing and Billing,Premium features
3,1085,Female,S,2,42.00,Inactive,No,Based on the customer service personnel I enco...,0,Service: Attitude,On-demand pickup location
4,0,Female,M,2,44.10,Active,No,Provide more convenient car pickup from the ai...,0,Service: Orders/Contracts,On-demand pickup location


In [8]:
test_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2771,Female,M,2,49.99,Inactive,No,"last time I rented a car was at Manchester, NH...",0,Product: Functioning,On-demand pickup location
1,1133,Male,S,1,56.05,Inactive,No,Please lower the prices.,0,Product: Pricing and Billing,Free Upgrade
2,900,Female,M,1,64.64,Active,No,Excellent response dealing with child seat.,1,Service: Accessibility,NaN
3,3795,Male,M,0,46.51,Inactive,No,"all went quite smoothly... it was Enterprise, ...",1,Service: Accessibility,NaN
4,3541,Male,S,1,17.01,Inactive,Yes,"Slow, long lineup",0,Product: Functioning,On-demand pickup location


<a id="models"></a>
## Foundation Models on Watsonx



Below code invokes Watson Machine Learning API to invoke Watsonx.ai LLMs


In [10]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

<a id="predict"></a>
## Evaluate the model, prompt and parameters

### **1. Customer satisfaction**

Define instructions for the model to recognize if customer was satisfied or unsatisfied.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [17]:
satisfaction_instruction = """

Decide if customer was satisfied or not based on the given feedback by customer. Respond 1 if satisfied and 0 if unsatisfied.

Customer service was friendly and helpful.

1

I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.

0

"""

In [34]:
train_df_new = train_data.sample(frac = 1).reset_index().head(20)
prompt_input = """Decide if customer was satisfied or not based on the given feedback by customer. Respond 1 if satisfied and 0 if unsatisfied.\n"""

for index, row in train_df_new.iterrows():  
    prompt_input = prompt_input +"""\n\nReview:\n{}\n\nClassification:\n{}\n""".format(row["Customer_Service"], row["Satisfaction"])

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [16]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 1,
    "min_new_tokens": 1,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v1"

Analyze the customer satisfaction for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [23]:
prompt = Prompt(access_token, project_id)

In [36]:
import numpy as np
test_data["Predicted_Satisfaction"] = 0
for index, row in test_data.iterrows():
    #test = test + """\nReview:\n{}\n\nClassification:\n""".format(row["Customer_Service"])
    generated_response = prompt.generate(prompt_input + """\nReview:\n{}\n\nClassification:\n""".format(row["Customer_Service"]), model_id, parameters)
    test_data.at[index, "Predicted_Satisfaction"] = generated_response
    

test_data["Predicted_Satisfaction"] = np.where(test_data["Predicted_Satisfaction"]== '', 0, test_data["Predicted_Satisfaction"])

### Calculate the F1 micro score

In [37]:
from sklearn.metrics import f1_score

print('f1_micro_score', f1_score(test_data["Satisfaction"].astype(int), test_data["Predicted_Satisfaction"].astype(int), average='micro'))

f1_micro_score 0.9


### **2. Offer Recommendation**

Define instructions for the model to recommend best offer to an unsatisfied customer.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [38]:
offer_recommendation_instruction = """Generate next best offer to unsatisfied customer. Choose offer recommendation from the following list: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.

I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.

Premium features

Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.

On-demand pickup location

VERY slow service!

Free Upgrade

It was absolutely ATROCIOUS! My wife and I were in a foreign country  when we realized that our car had an expired license plate and expired proof of insurance!

Voucher

Provide more convenient car pickup from the airport parking.

On-demand pickup location

They could really try work harder.

Free Upgrade

I had to wait in line for a long time to get and return the vehicle.  Also, the car was not clean.

Voucher

I would like the reps be knowledgeable about the immediate area around the rental agency and or have maps for the area available free of charge.

Premium features\n\n
"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [39]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 30,
    "min_new_tokens": 1,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v1"

Filter test data for unsatisfied customer

In [46]:
train_df_new = train_data[train_data['Satisfaction']== 0].sample(frac = 1).reset_index().head(20)
prompt_input = """Generate next best offer to unsatisfied customer. Choose offer recommendation from the following list: 'On-demand pickup location', 'Free Upgrade', 'Voucher', 'Premium features'.\n"""

for index, row in train_df_new.iterrows():  
    prompt_input = prompt_input +"""\n\nReview:\n{}\n\nAction:\n{}\n""".format(row["Customer_Service"], row["Action"])

In [47]:
prompt = Prompt(access_token, project_id)

In [48]:
import numpy as np

test_data["Predicted_Action"] = 0
for index, row in test_data.loc[test_data['Satisfaction'] == 0].iterrows():
    #test = test + """\nReview:\n{}\n\nClassification:\n""".format(row["Customer_Service"])
    generated_response = prompt.generate(prompt_input + """\nReview:\n{}\n\nAction:\n""".format(row["Customer_Service"]), model_id, parameters)
    test_data.at[index, "Predicted_Action"] = generated_response

In [50]:
#test_data[["Action", "Predicted_Action"]].head()

Analyze the recommended actions for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [53]:
test_data["Action"] = np.where(test_data["Action"].isnull(), 0, test_data["Action"])
test_data["Predicted_Action"] = np.where(test_data["Predicted_Action"].isnull(), 0, test_data["Predicted_Action"])

### Calculate the F1 micro score

In [54]:
from sklearn.metrics import f1_score

print('f1_micro_score', f1_score(test_data["Action"].astype(str), test_data["Predicted_Action"].astype(str), average='micro'))

f1_micro_score 0.82


---

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.